## Import library

In [2]:
import numpy as np
import pandas as pd
import random

## ทำการดึง dataset

In [3]:
data = pd.read_csv("cheese_encode_5.csv")
data.head()

,cheese,milk,country,family,type,fat_content,texture,rind,color,flavor,aroma,vegetarian,vegan
0,2,4,72,3,47,0,11,10,16,596,49,0,0
1,3,18,35,3,39,0,114,7,16,69,226,1,0
2,9,4,31,3,38,0,99,7,11,470,0,1,0
3,11,4,35,3,39,0,93,7,11,488,57,0,0
4,15,18,74,3,21,0,199,7,15,71,158,1,0


## Node Class
           Node Class เป็นส่วนสำคัญในการเก็บข้อมูลสำหรับแต่ละ "โหนด" (Node) ของต้นไม้ ซึ่งแต่ละโหนดจะเป็นจุดที่เราทำการตัดสินใจว่าจะไปในทิศทางไหนต่อจากข้อมูลที่มี โดยใช้ฟีเจอร์และ threshold ในการกำหนด

In [4]:
# Decision Tree Node Class (Unchanged)
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        self.feature_index = feature_index # กำหนดอินเด็กซ์ของฟีเจอร์ที่ใช้ในการแบ่งข้อมูล
        self.threshold = threshold # กำหนดค่าขีดจำกัดในการแบ่งข้อมูลของฟีเจอร์นั้น
        self.left = left  # โหนดซ้าย (subtree ซ้าย)
        self.right = right  # โหนดขวา (subtree ขวา)
        self.info_gain = info_gain # ค่าข้อมูลที่ได้รับจากการแบ่ง (information gain)
        
        self.value = value # ค่าของโหนด (ใช้ใน leaf nodes)

## Decision Tree Classifier
        ทำหน้าที่ในการสร้างและใช้งาน Decision Tree
### ภาพรวมการทำงานมี 3 ส่วน ที่สำคัญดังนี้
1. สร้างต้นไม้: ฟังก์ชัน build_tree จะสร้างต้นไม้ตัดสินใจจากข้อมูลโดยใช้กระบวนการค้นหาการแบ่งที่ดีที่สุดในแต่ละโหนด
2. การแบ่งข้อมูล: ฟังก์ชัน get_best_split ค้นหาการแบ่งที่เหมาะสมที่สุดโดยเลือกฟีเจอร์และค่าเกณฑ์ที่ให้ค่า Information Gain สูงสุด
3. ทำนายค่า: ฟังก์ชัน predict และ make_prediction ทำหน้าที่ทำนายค่าจากข้อมูลใหม่ โดยใช้โหนดต่าง ๆ ที่ถูกสร้างขึ้นจากต้นไม้

In [5]:
# Decision Tree Classifier (Unchanged)
class DecisionTreeClassifier():
    #ฟังก์ชันนี้เป็นการกำหนดค่าพื้นฐานของคลาส เช่น จำนวนตัวอย่างข้อมูลขั้นต่ำที่ใช้ในการแบ่ง 
    # (min_samples_split) และความลึกสูงสุดของต้นไม้ (max_depth)
    def __init__(self, min_samples_split=2, max_depth=2):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
    
    # เป็นฟังก์ชันที่สร้างต้นไม้โดยใช้การแบ่งข้อมูลแบบ recursive(ซั้า)
    def build_tree(self, dataset, curr_depth=0):
        X, Y = dataset[:, :-1], dataset[:, -1] # แยกข้อมูลออกเป็นฟีเจอร์ (X) และเป้าหมาย (Y)
        num_samples, num_features = np.shape(X) 

        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            # ค้นหาเกณฑ์ที่ดีที่สุด
            best_split = self.get_best_split(dataset, num_samples, num_features)

            # เมื่อได้ค่าที่ดีที่สุดแล้วจะสร้างโหนดซ้าย-ขวาจนถึงความลึกสูงสุดของ info_gain
            if best_split and best_split["info_gain"] > 0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth + 1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth + 1)
                return Node(best_split["feature_index"], best_split["threshold"], 
                      left_subtree, right_subtree, best_split["info_gain"])

        # กรณีไม่สามารถแบ่งได้แล้ว(best_split < 0) จะคำนวณค่าที่ดีที่สุดของ leaf node
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)

    # หาการแบ่งที่ดีที่สุดโดยใช้การคำนวณค่าข้อมูลที่ได้รับ (information gain) ซึ่งขึ้นอยู่กับเกณฑ์ Gini index หรือ entropy
    def get_best_split(self, dataset, num_samples, num_features):
    
        # เตรียมการเก็บค่าการแบ่งที่ดีที่สุด
        best_split = {}
        max_info_gain = -float("inf")
    
        # วนลูปผ่านทุกฟีเจอร์ในข้อมูล เพื่อทำการตรวจสอบว่าการแบ่งตามฟีเจอร์นั้นๆ จะให้ผลลัพธ์ที่ดีหรือไม่
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
        
            # วนลูปผ่านค่าขีดจำกัดที่เป็นไปได้ ผ่านแต่ละ threshold ที่เป็นไปได้สำหรับฟีเจอร์นั้นๆ
            for threshold in possible_thresholds:
                # dataset_left: ข้อมูลที่มีค่าน้อยกว่าหรือเท่ากับ threshold
                # dataset_right: ข้อมูลที่มีค่ามากกว่า threshold
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
            
                # ตรวจสอบว่าการแบ่งมีข้อมูลทั้งสองด้านหรือไม่
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                
                    # คำนวณค่า information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                
                    # อัปเดตการแบ่งที่ดีที่สุดหากค่าการเพิ่มข้อมูลสูงขึ้น
                    if curr_info_gain > max_info_gain:
                        best_split = {
                            "feature_index": feature_index,
                            "threshold": threshold,
                            "dataset_left": dataset_left,
                            "dataset_right": dataset_right,
                            "info_gain": curr_info_gain
                        }
                        max_info_gain = curr_info_gain

        # คืนค่าการแบ่งที่ดีที่สุด
        return best_split if "info_gain" in best_split else None

    # ฟังก์ชันนี้ใช้แบ่งข้อมูลออกเป็นสองส่วนตามฟีเจอร์และค่า threshold ที่กำหนด 
    # โดยข้อมูลที่ค่าฟีเจอร์น้อยกว่าหรือเท่ากับ threshold จะไปอยู่ที่ฝั่งซ้าย และข้อมูลที่มากกว่า threshold จะอยู่ฝั่งขวา
    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
        return dataset_left, dataset_right

    # ฟังก์ชันนี้คำนวณค่า Information Gain ที่ได้จากการแบ่งข้อมูล ซึ่งสามารถใช้เกณฑ์ Gini หรือ Entropy
    def information_gain(self, parent, l_child, r_child, mode="gini"):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode == "gini":
            gain = self.gini_index(parent) - (weight_l * self.gini_index(l_child) + weight_r * self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l * self.entropy(l_child) + weight_r * self.entropy(r_child))
        return gain

    # ฟังก์ชันนี้คำนวณค่า Gini impurity ซึ่งเป็นวิธีในการวัดการกระจายของข้อมูลในแต่ละโหนด
    # ยิ่งตํ่ายิ่งดี
    def gini_index(self, y):
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini

    # เมื่อไม่สามารถแบ่งข้อมูลได้อีกแล้ว ฟังก์ชันนี้จะคำนวณค่าทำนายสำหรับ leaf node 
    # โดยใช้ข้อมูลส่วนใหญ่ในโหนดนั้นในการกำหนดค่าทำนาย
    def calculate_leaf_value(self, Y):
        Y = list(Y)
        return max(Y, key=Y.count)

    # ฟังก์ชันนี้เป็นการรวมข้อมูลฟีเจอร์และ target เข้าด้วยกัน
    def fit(self, X, Y):
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)

    # ฟังก์ชันนี้ใช้ในการทำนายค่าจากข้อมูลใหม่ 
    #โดยผ่านการวนลูปที่โหนดในต้นไม้โดยใช้ฟังก์ชัน make_prediction เพื่อหาค่าทำนายในแต่ละจุด
    def predict(self, X):
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions

    # ฟังก์ชันนี้ใช้ในการทำ recursive เพื่อค้นหาโหนดที่เป็น leaf node และส่งค่าทำนายกลับมาเมื่อถึง leaf node 
    def make_prediction(self, x, tree):
        if tree.value != None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

## Random Forest Classifier Implementation
         ในส่วนนี้ทำการ implement Random Forest Classifier โดยมีการสร้างต้นไม้หลายต้น (Decision Trees) และใช้การทำนายผลจากแต่ละต้นไม้ร่วมกันเพื่อให้ผลลัพธ์สุดท้ายออกมาดีขึ้น
### ภาพรวมการทำงานมี 2 ส่วน ที่สำคัญดังนี้
1. สร้าง Random Foresrt classifier: ฟังก์ชัน fit สร้างต้นไม้ตัดสินใจหลายต้นตามจำนวนที่กำหนด (n_estimators)
2. ทำนายค่า: ฟังก์ชัน predict ใช้การทำนายจากต้นไม้ทุกต้นในป่า และใช้วิธี majority vote เพื่อเลือกค่าที่เป็นคำตอบสุดท้าย

In [6]:
# Random Forest Classifier Implementation
class RandomForestClassifierFromScratch:
    def __init__(self, n_estimators=10, max_depth=5, min_samples_split=2, max_features='sqrt'):
        self.n_estimators = n_estimators # จำนวนต้นไม้ที่จะสร้างใน Random Forest
        self.max_depth = max_depth # ความลึกสูงสุดของแต่ละต้นไม้
        self.min_samples_split = min_samples_split # จำนวนตัวอย่างข้อมูลขั้นต่ำที่ใช้ในการแบ่งโหนด
        self.max_features = max_features # จำนวนฟีเจอร์สูงสุด
        self.trees = []

    # ฟังก์ชันนี้ใช้ในการสร้าง Random Forest จาก Train set
    def fit(self, X, Y):
        self.trees = []
        for _ in range(self.n_estimators):
            X_sample, Y_sample = self._bootstrap_sampling(X, Y)
            tree = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X_sample, Y_sample)
            self.trees.append(tree)

    # ฟังก์ชันนี้ใช้สำหรับการสุ่มข้อมูลแบบ bootstrap
    # bootstrap sampling ซึ่งเป็นการสุ่มข้อมูลแบบมีการทดแทน (replace=True) 
    # หมายความว่าตัวอย่างข้อมูลเดิมสามารถถูกสุ่มซ้ำได้
    def _bootstrap_sampling(self, X, Y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, size=n_samples, replace=True)
        return X[indices], Y[indices]

    # ฟังก์ชันนี้ใช้สำหรับทำนายค่าจากข้อมูลใหม่
    def predict(self, X):
        tree_predictions = np.array([tree.predict(X) for tree in self.trees])
        final_predictions = [self._majority_vote(tree_predictions[:, i]) for i in range(X.shape[0])]
        return final_predictions

    # ฟังก์ชันนี้ใช้สำหรับหาคำตอบที่ได้รับการโหวตมากที่สุดจากผลการทำนายของต้นไม้หลายต้น
    def _majority_vote(self, predictions):
        return max(set(predictions), key=list(predictions).count)

## ทำการแยกข้อมูลเป็น feature และ target

In [7]:
X = data.drop('family', axis=1).values
Y = data['family'].values.reshape(-1, 1)

print(X)
print(Y)

[[  2   4  72 ...  49   0   0]
 [  3  18  35 ... 226   1   0]
 [  9   4  31 ...   0   1   0]
 ...
 [693  13   4 ...  87   1   0]
 [873   4  14 ... 250   0   0]
 [832  13  35 ... 181   0   0]]
[[ 3]
 [ 3]
 [ 3]
 ...
 [12]
 [12]
 [12]]


## ทำการแบ่งชุดข้อมูล train set และ test set

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

## ทำการสร้างโมเดล Random Forest
- กำหนดไห้สร้างต้นไม้ 100 ต้น
- ความลึกสูงสุดของต้นไม้เป็น 12 ชั้น
- กำหนด sample ขั้นตํ่าที่ใช้ในการแยกข้อมูล 3 ตัวอย่าง

In [9]:
# Create Random Forest Model
rf_classifier = RandomForestClassifierFromScratch(n_estimators=100, max_depth=12, min_samples_split=3)
rf_classifier.fit(X_train, Y_train)

In [10]:
# ทำการเก็บผลลัพธ์ที่คาดการณ์จากโมเดล Random Forest
Y_pred_rf = rf_classifier.predict(X_test)

## ทำการคำนวณ accuracy ของโมเดล Random Forest

In [11]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(Y_test, Y_pred_rf)
print(f"Random Forest Model Accuracy: {accuracy}")

Random Forest Model Accuracy: 0.9126117179741807


In [12]:
print(Y_pred_rf[:])

[12, 0, 4, 10, 3, 2, 11, 0, 10, 7, 12, 2, 3, 9, 8, 7, 7, 1, 3, 1, 10, 4, 5, 3, 12, 6, 0, 3, 4, 12, 2, 1, 10, 1, 10, 12, 11, 5, 8, 6, 11, 0, 11, 10, 6, 3, 12, 2, 9, 10, 0, 9, 6, 0, 1, 2, 10, 1, 6, 1, 4, 8, 4, 7, 6, 3, 11, 3, 3, 4, 12, 5, 9, 3, 8, 1, 12, 5, 8, 6, 11, 0, 6, 10, 9, 10, 0, 10, 10, 3, 5, 8, 8, 6, 0, 11, 11, 8, 11, 1, 2, 9, 9, 1, 5, 8, 4, 8, 0, 5, 10, 1, 1, 2, 11, 10, 11, 6, 3, 12, 11, 3, 7, 6, 3, 4, 3, 8, 4, 0, 1, 7, 7, 9, 10, 3, 1, 3, 3, 3, 12, 9, 0, 12, 3, 3, 2, 0, 4, 2, 3, 6, 1, 8, 9, 1, 10, 6, 12, 5, 6, 0, 1, 1, 3, 8, 0, 12, 7, 2, 10, 8, 3, 0, 3, 9, 5, 9, 7, 11, 3, 10, 9, 4, 5, 9, 1, 3, 8, 2, 8, 3, 11, 12, 5, 8, 7, 9, 0, 9, 1, 3, 2, 7, 11, 4, 10, 2, 12, 12, 1, 6, 9, 0, 0, 6, 3, 10, 5, 9, 11, 9, 2, 8, 4, 5, 4, 4, 5, 10, 12, 2, 1, 6, 3, 1, 0, 8, 2, 7, 4, 6, 11, 3, 2, 0, 4, 9, 5, 3, 4, 3, 3, 9, 7, 0, 2, 5, 1, 7, 9, 5, 3, 1, 2, 10, 9, 3, 9, 8, 0, 10, 8, 2, 7, 3, 12, 11, 1, 0, 11, 8, 7, 7, 0, 1, 3, 9, 1, 5, 11, 11, 12, 4, 11, 9, 2, 2, 3, 3, 11, 8, 12, 5, 7, 8, 3, 4, 12, 10, 1

In [13]:
print(X_test[:,:])

[[ 359    4   77 ...  127    1    0]
 [1068    4   77 ...  100    1    0]
 [  71   13   35 ...  105    0    0]
 ...
 [ 653    4   52 ...  181    0    0]
 [ 301   13   35 ...    0    0    0]
 [1071    4   77 ...  268    1    0]]


In [14]:
print(Y_test)

[[12]
 [ 0]
 [ 4]
 ...
 [ 0]
 [ 1]
 [ 0]]


## สร้างและแสดง Classification Report

In [15]:
from sklearn.metrics import classification_report

report = classification_report(Y_test, Y_pred_rf)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.61      0.69        95
           1       0.79      0.93      0.86        70
           2       1.00      1.00      1.00        78
           3       0.86      0.73      0.79       118
           4       1.00      1.00      1.00        69
           5       0.99      1.00      0.99        68
           6       0.85      0.91      0.88        66
           7       0.98      0.97      0.98        67
           8       0.90      0.96      0.93        74
           9       0.99      0.97      0.98        80
          10       0.95      0.99      0.97        80
          11       0.86      1.00      0.93        68
          12       0.95      1.00      0.97        74

    accuracy                           0.91      1007
   macro avg       0.92      0.93      0.92      1007
weighted avg       0.91      0.91      0.91      1007



## การบันทึกโมเดลที่สร้างขึ้น

In [16]:
import joblib
joblib.dump(rf_classifier,'model_now.joblib')

['model_now.joblib']